In [ ]:
import random
import subprocess
import itertools
from collections import defaultdict
import importlib.machinery
import os
import time

import matplotlib.pyplot as plt
import numpy as np
%config InlineBackend.figure_format='retina'

In [ ]:
labels = []
sizes = defaultdict(lambda: [])
times = defaultdict(lambda: [])

from sysconfig import get_paths as gp
suffix = importlib.machinery.EXTENSION_SUFFIXES[0]
pybind11_path = '/Users/wjakob/pybind11/include'
boost_path = '/Users/wjakob/nanobind/bench/boost_1_78_0'
cmd_base = ['clang++', '-mcpu=apple-a14', '-shared', '-undefined', 'dynamic_lookup',
            '-rpath', '..', '-std=c++17', '-I', '../include', '-I', gp()['include'],
            '-I', pybind11_path, '-I', boost_path, '-Wno-deprecated-declarations',
            f'-L{boost_path}/stage/lib', '-L..', '-rpath', f'{boost_path}/stage/lib', '-fno-stack-protector']



def gen_file(name, func, libs=('boost', 'pybind11', 'nanobind')):
    for i, lib in enumerate(libs):    
        for opt_mode, opt_flags in {'debug' : ['-O0', '-g3'], 'opt' : ['-Os', '-g0']}.items():
            with open(name + '_' + lib + '.cpp', 'w') as f:
                if lib == 'boost':
                    f.write(f'#include <boost/python.hpp>\n')
                    f.write(f'namespace py = boost::python;\n\n')
                    f.write(f'BOOST_PYTHON_MODULE({name}_{lib}_{opt_mode}) {{\n')
                else:
                    f.write(f'#include <{lib}/{lib}.h>\n\n')

                    f.write(f'namespace py = {lib};\n\n')

                    prefix = "NB" if lib == 'nanobind' else "PYBIND11"
                    f.write(f'{prefix}_MODULE({name}_{lib}_{opt_mode}, m) {{\n')

                func(f, lib)
                f.write(f'}}\n')

            fname_out = name + '_' + lib + '_' + opt_mode  + suffix
            cmd = cmd_base + opt_flags + [name + '_' + lib + '.cpp', '-o', fname_out]
            if lib == 'nanobind':
                cmd += ['-lnanobind']
            elif lib == 'boost':
                cmd += ['-lboost_python39']
                
            print(' '.join(cmd))
            time_before = time.perf_counter()
            subprocess.check_call(cmd)
            time_after = time.perf_counter()
            if opt_mode != 'debug':
                subprocess.check_call(['strip', '-x', fname_out])
            if i == 0:
                labels.append(name + ' [' + opt_mode + ']')
            sizes[lib].append(os.path.getsize(fname_out) / (1024 * 1024))
            times[lib].append(time_after-time_before)


            
def gen_func(f, lib):
    types = [ 'uint16_t', 'int32_t', 'uint32_t', 'int64_t', 'uint64_t', 'float' ]
    if lib == 'boost':
        prefix = 'py::'
    else:
        prefix = 'm.'
    for i, t in enumerate(itertools.permutations(types)):
        args = f'{t[0]} a, {t[1]} b, {t[2]} c, {t[3]} d, {t[4]} e, {t[5]} f'
        f.write('    %sdef("test_%04i", +[](%s) { return a+b+c+d+e+f; });\n' % (prefix, i, args))


def gen_class(f, lib):
    types = [ 'uint16_t', 'int32_t', 'uint32_t', 'int64_t', 'uint64_t', 'float' ]

    for i, t in enumerate(itertools.permutations(types)):
        if lib == 'boost':
            prefix = ''
            postfix = f', py::init<{t[0]}, {t[1]}, {t[2]}, {t[3]}, {t[4]}, {t[4]}>()'
            func_prefix = 'py::def'

        else:
            prefix = 'm, '
            postfix = ''
            func_prefix = 'm.def'

       
        f.write(f'    struct Struct{i} {{\n')
        f.write(f'        {t[0]} a; {t[1]} b; {t[2]} c; {t[3]} d; {t[4]} e; {t[5]} f;\n')
        f.write(f'        Struct{i}({t[0]} a, {t[1]} b, {t[2]} c, {t[3]} d, {t[4]} e, {t[5]} f) : a(a), b(b), c(c), d(d), e(e), f(f) {{ }}\n')
        f.write(f'        float sum() const {{ return a+b+c+d+e+f; }}\n')
        f.write(f'    }};\n')
        f.write(f'    py::class_<Struct{i}>({prefix}\"Struct{i}\"{postfix})\n')
        if lib != 'boost':
                f.write(f'        .def(py::init<{t[0]}, {t[1]}, {t[2]}, {t[3]}, {t[4]}, {t[5]}>())\n')
        f.write(f'        .def("sum", &Struct{i}::sum);\n\n')
        
        if i > 250:
            break;
        
        
gen_file('func', gen_func)
gen_file('class', gen_class)


print(labels)
print(dict(sizes))
print(dict(times))

In [ ]:
x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars

fig, ax = plt.subplots(figsize=[11.25, 3])
rects0 = ax.bar(x - width, times['boost'], width, label='boost', align='center', edgecolor='black')
rects1 = ax.bar(x, times['pybind11'], width, label='pybind11', align='center', edgecolor='black')
rects2 = ax.bar(x + width, times['nanobind'], width, label='nanobind', align='center', edgecolor='black')

ax.set_ylabel('Time (seconds)')
ax.set_title('Compilation time')
ax.set_xticks(x, labels)
ax.legend()
ylim = np.max(times['boost'])* .76
ax.set_ylim(
    0, ylim
)

def adj(ann):
    for a in ann:
        if a.xy[1] > ylim*.9:
            a.xy = (a.xy[0], ylim * 0.8)
            a.set_color('white')


improvement = np.array(times['boost']) / np.array(times['nanobind'])
improvement = ['%.2f\n(x %.1f)' % (times['boost'][i], v) for i, v in enumerate(improvement)]
adj(ax.bar_label(rects0, labels=improvement, padding=3))

improvement = np.array(times['pybind11']) / np.array(times['nanobind'])
improvement = ['%.2f\n(x %.1f)' % (times['pybind11'][i], v) for i, v in enumerate(improvement)]
adj(ax.bar_label(rects1, labels=improvement, padding=3))

adj(ax.bar_label(rects2, fmt='%.2f'))

fig.tight_layout()
plt.savefig('times.png', facecolor='white', dpi=200)
plt.savefig('times.svg', facecolor='white')
plt.show()

In [ ]:
x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars

fig, ax = plt.subplots(figsize=[11.25, 3])
rects0 = ax.bar(x - width, sizes['boost'], width, label='boost', align='center', edgecolor='black')
rects1 = ax.bar(x, sizes['pybind11'], width, label='pybind11', align='center', edgecolor='black')
rects2 = ax.bar(x + width, sizes['nanobind'], width, label='nanobind', align='center', edgecolor='black')

ax.set_ylabel('Size (MiB)')
ax.set_title('Binary size')
ax.set_xticks(x, labels)
ax.legend()
ylim = np.max(sizes['boost'])* .2
ax.set_ylim(
    0, ylim
)

def adj(ann):
    for a in ann:
        if a.xy[1] > ylim:
            a.xy = (a.xy[0], ylim * 0.8)
            a.set_color('white')


improvement = np.array(sizes['boost']) / np.array(sizes['nanobind'])
improvement = ['%.2f\n(x %.1f)' % (sizes['boost'][i], v) for i, v in enumerate(improvement)]
adj(ax.bar_label(rects0, labels=improvement, padding=3))

improvement = np.array(sizes['pybind11']) / np.array(sizes['nanobind'])
improvement = ['%.2f\n(x %.1f)' % (sizes['pybind11'][i], v) for i, v in enumerate(improvement)]
adj(ax.bar_label(rects1, labels=improvement, padding=3))

adj(ax.bar_label(rects2, fmt='%.2f'))

fig.tight_layout()
plt.savefig('sizes.png', facecolor='white', dpi=200)
plt.savefig('sizes.svg', facecolor='white')
plt.show()

In [ ]:
labels2=[]

class native_module:
    @staticmethod
    def test_0000(a, b, c, d, e, f):
        return a + b + c + d +e + f

    
    class Struct0:
        def __init__(self, a, b, c, d, e, f):
            self.a = a
            self.b = b
            self.c = c
            self.d = d
            self.e = e
            self.f = f

        def sum(self):
            return self.a + self.b + self.c + self.e + self.f
    


rtimes = defaultdict(lambda: [])
for name in ['func', 'class']:
    its = 1000000 if name == 'func' else 500000
    for lib in ['python', 'pybind11', 'nanobind', 'boost']:
        for mode in ['debug', 'opt']:
            if lib == 'python':
                m = native_module
            else:
                m = importlib.import_module(f'{name}_{lib}_{mode}')
         
            time_before = time.perf_counter()
            if name == 'func':
                for i in range(its):
                    m.test_0000(1,2,3,4,5,6)
            elif name == 'class':
                for i in range(its):
                    m.Struct0(1,2,3,4,5,6).sum()

            time_after = time.perf_counter()

            if lib == 'pybind11':
                labels2.append(name + '_' + mode)
            rtimes[lib].append(time_after-time_before)

In [ ]:
x = np.arange(len(labels))  # the label locations
width = 0.22  # the width of the bars

import matplotlib as mpl
mpl.rcParams['hatch.linewidth'] = 5.0 

fig, ax = plt.subplots(figsize=[11.25, 3])
rects1 = ax.bar(x- 1.5*width, rtimes['boost'], width, label='boost', align='center', edgecolor='black')
rects2 = ax.bar(x - width/2, rtimes['pybind11'], width, label='pybind11', align='center', edgecolor='black')
rects3 = ax.bar(x + width/2, rtimes['nanobind'], width, label='nanobind', align='center', edgecolor='black')
rects0 = ax.bar(x + 1.5*width, rtimes['python'], width, label='python', align='center', hatch="/", edgecolor='white')
ax.bar(x + 1.5*width, rtimes['python'], width, align='center', edgecolor='black', facecolor='None')


ax.set_ylabel('Time (seconds)')
ax.set_title('Runtime performance')
ax.set_xticks(x, labels)
ax.legend()
ylim = np.max(rtimes['pybind11'])* .32
ax.set_ylim(0, ylim)

def adj(ann):
    for a in ann:
        if a.xy[1] > ylim:
            a.xy = (a.xy[0], ylim * 0.8)
            a.set_color('white')


improvement = np.array(rtimes['python']) / np.array(rtimes['nanobind'])
improvement = ['%.2f\n(x %.1f)' % (rtimes['python'][i], v) for i, v in enumerate(improvement)]
adj(ax.bar_label(rects0, labels=improvement, padding=3))

improvement = np.array(rtimes['boost']) / np.array(rtimes['nanobind'])
improvement = ['%.2f\n(x %.1f)' % (rtimes['boost'][i], v) for i, v in enumerate(improvement)]
adj(ax.bar_label(rects1, labels=improvement, padding=3))

improvement = np.array(rtimes['pybind11']) / np.array(rtimes['nanobind'])
improvement = ['%.2f\n(x %.1f)' % (rtimes['pybind11'][i], v) for i, v in enumerate(improvement)]
adj(ax.bar_label(rects2, labels=improvement, padding=3))

adj(ax.bar_label(rects3, fmt='%.2f'))

fig.tight_layout()
plt.savefig('perf.png', facecolor='white', dpi=200)
plt.savefig('perf.svg', facecolor='white')
plt.show()